In [ ]:
# always check your python version
!python --version

In [ ]:
# AWS SDK for Python, -q flag is for quiet
%pip install -q boto3

In [ ]:
# always check your boto3 version
%pip show boto3

In [ ]:
!sudo apt-get install sqlite3 -y

In [ ]:
!sqlite3 --version

In [ ]:
from chat_history import Chat
from helper import load_text_file
import json
import boto3
client = boto3.client('bedrock-runtime')

In [ ]:
def converse(session_id, input):
  history = Chat.read(session_id)  

  system_prompt = load_text_file("prompts/jp-teacher.txt")
  model_id = 'cohere.command-r-v1:0'
  prompt = f"{system_prompt}\n{history}\nUser: ${input}\nAssistant:"
  resp = client.invoke_model(
      modelId=model_id,
      contentType='application/json',
      accept='application/json',
      body=json.dumps({
        "message": prompt,
        "max_tokens": 200,
        "temperature": 1.0
      })
  )
  body = json.loads(resp.get("body").read())
  llm_response = body['text']

  updated_history = f"{history}\n{llm_response}"
  Chat.write(session_id,updated_history)

  return body

In [ ]:
print(converse("food","Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

In [ ]:
print(converse("How do I conjugate taberu?"))

In [ ]:
class JpAgent:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [] # load with initial messages
  def chat(self, text):
    self.messages.append({"role": "user", "content": [{"text": text}]})
    system_prompt_text = load_text_file("prompts/jp-teacher.txt")
    system_prompt = [{"text": system_prompt_text}]
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse(
        modelId=model_id,
        messages=self.messages,
        system=system_prompt
    )
    return resp
agent = JpAgent()

In [ ]:
print(agent.chat("Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

In [ ]:
print(agent.chat("How do I conjugate taberu?"))